# Municipal Population Dataset (1996–2024): Cleaning and Standarization
## *Preprocessing workflow for INE municipal time-series data*🧹

In [1]:
"""
Notebook: 01_data_cleaning_padron_historico.ipynb
Author: Juan Zotes
Created during exploratory phase (Dec 2025)
Last updated: 2026-02-05


Purpose:
    Clean and standardize historical municipal census data (Padrón Municipal)
    for Spain (1996–2024), ensuring consistency across years and municipalities.

Scope:
    - Input: Raw INE historical census CSV files
    - Output: Cleaned, harmonized demographic dataset
    - Spatial unit: Municipality (INE codes)

Notes:
    - This notebook focuses exclusively on data cleaning and preprocessing
    - Analytical steps are handled in subsequent notebooks
    - Additional demographic variables may be integrated in future revisions

Status:
    - Core cleaning pipeline implemented
    - Structure considered stable
    - Content may be extended if new raw variables are added
"""


'\nNotebook: 01_data_cleaning_padron_historico.ipynb\nAuthor: Juan Zotes\nCreated during exploratory phase (Dec 2025)\nLast updated: 2026-02-02\n\n\nPurpose:\n    Clean and standardize historical municipal census data (Padrón Municipal)\n    for Spain (1996–2024), ensuring consistency across years and municipalities.\n\nScope:\n    - Input: Raw INE historical census CSV files\n    - Output: Cleaned, harmonized demographic dataset\n    - Spatial unit: Municipality (INE codes)\n\nNotes:\n    - This notebook focuses exclusively on data cleaning and preprocessing\n    - Analytical steps are handled in subsequent notebooks\n    - Additional demographic variables may be integrated in future revisions\n\nStatus:\n    - Core cleaning pipeline implemented\n    - Structure considered stable\n    - Content may be extended if new raw variables are added\n'

### 🗂️Import and read the files
Due to the structure of INE datasets, it is important to apply the processing steps defined below, as they make it easier to load any files downloaded from Spanish public institutions, which generally follow this same format.

In [2]:
# Import Libraries
from pathlib import Path
import pandas as pd

# Base data directory (portable across Windows, Linux, Codespaces)
DATA_DIR = Path(
    r"/workspaces/rural-migration-land-use-spain/data/demography/raw"
)

# File path
fp_padron_historico = DATA_DIR / "00_raw_padron_1996_2024.csv"

# Read / import file
padron_hist = pd.read_csv(
    fp_padron_historico,
    # INE downloads: CSV uses ";" as separator
    sep=";",
    # The INE uses "." as the thousands separator, so we define it here
    # to allow pandas to automatically parse numeric columns correctly.
    thousands=".",
    # "," is used as the decimal separator
    decimal=",",
    # Set the data type of each column in Python
    dtype={
        "Municipios": str,
        "Sexo": str,
        "Periodo": int,
        "Total": float
    },
    # low_memory=False ensures pandas reads whole columns to infer types reliably
    low_memory=False,
    # encoding="latin1" because INE CSV files typically use this encoding
    # and this avoids issues with accents/ñ during import
    encoding="latin1"
)

### Preview

In [3]:
padron_hist.head()

,ï»¿Municipios,Sexo,Periodo,Total
0,44001 Ababuj,Total,2024,74.0
1,44001 Ababuj,Total,2023,70.0
2,44001 Ababuj,Total,2022,72.0
3,44001 Ababuj,Total,2021,76.0
4,44001 Ababuj,Total,2020,77.0


In [4]:
padron_hist.tail()

,ï»¿Municipios,Sexo,Periodo,Total
708001,04103 Zurgena,Mujeres,2000,1064.0
708002,04103 Zurgena,Mujeres,1999,1068.0
708003,04103 Zurgena,Mujeres,1998,1069.0
708004,04103 Zurgena,Mujeres,1997,NaN
708005,04103 Zurgena,Mujeres,1996,1075.0


### 🔧Rename Columns

In [5]:
# Create a dictionnary with the new names
new_names = {"ï»¿Municipios": "Municipios",
             "Sexo": "Cat",
            "Periodo": "Year",
            "Total": "Pop"}

# Rename the columns
padron_hist = padron_hist.rename(columns=new_names)

# See Result
padron_hist.head()

,Municipios,Cat,Year,Pop
0,44001 Ababuj,Total,2024,74.0
1,44001 Ababuj,Total,2023,70.0
2,44001 Ababuj,Total,2022,72.0
3,44001 Ababuj,Total,2021,76.0
4,44001 Ababuj,Total,2020,77.0


### 🔧Split one column into two

In [6]:
# " ".split(" ", 1) → splits only at the first space
# expand=True → creates two new columns
padron_hist[["Mun_Code", "Mun"]] = padron_hist["Municipios"].str.split(" ", n=1, expand=True)

# See Result
padron_hist.head()

,Municipios,Cat,Year,Pop,Mun_Code,Mun
0,44001 Ababuj,Total,2024,74.0,44001,Ababuj
1,44001 Ababuj,Total,2023,70.0,44001,Ababuj
2,44001 Ababuj,Total,2022,72.0,44001,Ababuj
3,44001 Ababuj,Total,2021,76.0,44001,Ababuj
4,44001 Ababuj,Total,2020,77.0,44001,Ababuj


### 🔧Column Rearrangement

In [7]:
# Delete Municipios column
padron_hist = padron_hist.drop(columns=["Municipios"])

#Rearrange columns order
padron_hist = padron_hist[["Mun_Code", "Mun", "Cat", "Year", "Pop"]]

#See result
padron_hist.tail()

,Mun_Code,Mun,Cat,Year,Pop
708001,04103,Zurgena,Mujeres,2000,1064.0
708002,04103,Zurgena,Mujeres,1999,1068.0
708003,04103,Zurgena,Mujeres,1998,1069.0
708004,04103,Zurgena,Mujeres,1997,NaN
708005,04103,Zurgena,Mujeres,1996,1075.0


### 🚫Removing the Empty Census Year (1997)
The year 1997 does not contain any population data in the official INE records, as the municipal census was not conducted that year.
Keeping this empty year would introduce unnecessary NaN values, bias percentage-change calculations, and clutter the dataset without adding meaningful information.

For clarity, analytical consistency, and to ensure clean downstream processing, the entire year 1997 is removed from the cleaned dataset.
The raw dataset remains untouched, preserving the original record.

In [8]:
padron_hist = padron_hist[padron_hist["Year"] != 1997]

#See result
padron_hist.tail()

,Mun_Code,Mun,Cat,Year,Pop
708000,04103,Zurgena,Mujeres,2001,1062.0
708001,04103,Zurgena,Mujeres,2000,1064.0
708002,04103,Zurgena,Mujeres,1999,1068.0
708003,04103,Zurgena,Mujeres,1998,1069.0
708005,04103,Zurgena,Mujeres,1996,1075.0


### 🔍Data Format & Assertions

In [9]:
# Check Data types
padron_hist.dtypes

Mun_Code        str
Mun             str
Cat             str
Year          int64
Pop         float64
dtype: object

In [10]:
# Transform selected columns into appropriate data types
padron_hist = padron_hist.astype({
    # keep as string to preserve leading zeros
    "Mun_Code": "string",
    # municipalities names as strings
    "Mun":      "string",
    # category (Total, Hombres, Mujeres)
    "Cat":      "string",   
    # years stored as integers
    "Year":     "int64",    
    # Pop stays as float as defined earlier (float64)
})

# Check Data types
padron_hist.dtypes

Mun_Code     string
Mun          string
Cat          string
Year          int64
Pop         float64
dtype: object

In [12]:
# Assert that all population values are non-negative
assert (padron_hist["Pop"] >= 0).all(), "Population values must be >= 0"

AssertionError: Population values must be >= 0

In [13]:
# Check rows with NaN
padron_hist[padron_hist["Pop"].isna()]

,Mun_Code,Mun,Cat,Year,Pop
10716,10903,AlagÃ³n del RÃ­o,Total,2009,NaN
10717,10903,AlagÃ³n del RÃ­o,Total,2008,NaN
10718,10903,AlagÃ³n del RÃ­o,Total,2007,NaN
10719,10903,AlagÃ³n del RÃ­o,Total,2006,NaN
10720,10903,AlagÃ³n del RÃ­o,Total,2005,NaN
...,...,...,...,...,...
707826,48916,Usansolo,Mujeres,2001,NaN
707827,48916,Usansolo,Mujeres,2000,NaN
707828,48916,Usansolo,Mujeres,1999,NaN
707829,48916,Usansolo,Mujeres,1998,NaN


In [14]:
print(f"NaN values in Pop: {padron_hist['Pop'].isna().sum()}")

NaN values in Pop: 1860


In [15]:
# Assert that all non_NaN population values are non-negative
assert (padron_hist["Pop"].dropna() >= 0).all(), "Non-null population values must be >= 0"


In [16]:
print(padron_hist["Mun_Code"].nunique())
assert padron_hist["Mun_Code"].nunique() == 8132, "There should be 8132 unique municipalities in Spain"

8138


AssertionError: There should be 8132 unique municipalities in Spain

In [17]:
padron_hist.groupby("Year")["Mun_Code"].nunique()


Year
1996    8138
1998    8138
1999    8138
2000    8138
2001    8138
2002    8138
2003    8138
2004    8138
2005    8138
2006    8138
2007    8138
2008    8138
2009    8138
2010    8138
2011    8138
2012    8138
2013    8138
2014    8138
2015    8138
2016    8138
2017    8138
2018    8138
2019    8138
2020    8138
2021    8138
2022    8138
2023    8138
2024    8138
Name: Mun_Code, dtype: int64

### Data Cleaning: Ghost Codes and Historical Mergers

The INE padron (1996-2024) contains **8,138 municipality codes**, but the official
INE count for 2026 is **8,132**. This discrepancy of 6 codes is due to:

#### Ghost Codes (2 codes to remove)
Codes that never had population data — INE errors:

| Code | Name | Issue |
|------|------|-------|
| 12066 | Gatova | Error in INE. Real Gátova is 46902 (Valencia) |
| 17122 | Palmerola | Never was a municipality. Minor entity of Les Llosses (17096) |

#### Historical Mergers (4 codes to aggregate)
Municipalities that merged during the time series:

| Old Code | Old Name | New Code | New Name | Merge Year |
|----------|----------|----------|----------|------------|
| 15026 | Cesuras | 15902 | Oza-Cesuras | 2013 |
| 15063 | Oza dos Ríos | 15902 | Oza-Cesuras | 2013 |
| 36011 | Cerdedo | 36902 | Cerdedo-Cotobade | 2016 |
| 36012 | Cotobade | 36902 | Cerdedo-Cotobade | 2016 |

#### Approach
- **Ghost codes**: Remove entirely (no data loss)
- **Historical mergers**: Aggregate retrospectively to current municipality code

This ensures a continuous time series with **8,132 municipalities**, matching the
current INE official count. Historical granularity at sub-municipal level is lost,
but the dataset becomes consistent and directly comparable with current administrative
boundaries.

In [18]:
# Check ghost codes data
print("=== 12066 - Gatova ===")
print(padron_hist[(padron_hist['Mun_Code'] == 12066) & (padron_hist['Cat'] == 'Total')][['Year', 'Pop']])

print("\n=== 17122 - Palmerola ===")
print(padron_hist[(padron_hist['Mun_Code'] == 17122) & (padron_hist['Cat'] == 'Total')][['Year', 'Pop']])

=== 12066 - Gatova ===
Empty DataFrame
Columns: [Year, Pop]
Index: []

=== 17122 - Palmerola ===
Empty DataFrame
Columns: [Year, Pop]
Index: []


In [19]:
# ==============================================================================
# CLEAN PADRON: REMOVE GHOST CODES AND AGGREGATE HISTORICAL MERGERS
# ==============================================================================
# The INE padron (1996-2024) contains 8,138 municipality codes, but the official
# INE count for 2026 is 8,132. The difference of 6 codes includes:
#
# GHOST CODES (2) - Never had population data, should be removed:
#   - 12066 Gatova: Error in INE, real Gátova is 46902 (Valencia)
#   - 17122 Palmerola: Never was a municipality, minor entity of Les Llosses
#
# HISTORICAL MERGERS (4) - Aggregated retrospectively to current municipality:
#   - 15026 Cesuras      + 15063 Oza dos Ríos → 15902 Oza-Cesuras (merged 2013)
#   - 36011 Cerdedo      + 36012 Cotobade     → 36902 Cerdedo-Cotobade (merged 2016)
#
# Note: New codes (15902, 36902) already exist in padron from their merge year.
#       We aggregate old codes INTO existing new codes, then remove old codes.
#
# Final count: 8,138 - 2 (ghosts) - 4 (old merger codes) = 8,132
# ==============================================================================

print("\n" + "="*70)
print("CLEANING PADRON: GHOST CODES AND HISTORICAL MERGERS")
print("="*70)

# Define ghost codes AS STRINGS (Mun_Code is string type to preserve leading zeros)
GHOST_CODES = ['12066', '17122']

# Define historical mergers: new_code ← old_codes (ALL AS STRINGS)
HISTORICAL_MERGERS = {
    '15902': {
        'new_name': 'Oza-Cesuras',
        'merge_year': 2013,
        'old_codes': {
            '15026': 'Cesuras',
            '15063': 'Oza dos Ríos'
        }
    },
    '36902': {
        'new_name': 'Cerdedo-Cotobade',
        'merge_year': 2016,
        'old_codes': {
            '36011': 'Cerdedo',
            '36012': 'Cotobade'
        }
    }
}

count_before = padron_hist['Mun_Code'].nunique()
print(f"\nMunicipalities before cleaning: {count_before}")

# --- STEP 1: Remove ghost codes ---
print(f"\n" + "-"*70)
print(f"STEP 1: REMOVING GHOST CODES")
print("-"*70)

for code in GHOST_CODES:
    records = padron_hist[padron_hist['Mun_Code'] == code]
    if len(records) > 0:
        name = records['Mun'].iloc[0]
        print(f"   ✗ {code} - {name}: {len(records)} records removed")
    else:
        print(f"   ○ {code} - Not found (already removed or doesn't exist)")

padron_hist = padron_hist[~padron_hist['Mun_Code'].isin(GHOST_CODES)].copy()

# --- STEP 2: Aggregate historical mergers ---
print(f"\n" + "-"*70)
print(f"STEP 2: AGGREGATING HISTORICAL MERGERS")
print("-"*70)
print(f"Old municipality data is added to existing new municipality codes:\n")

for new_code, info in HISTORICAL_MERGERS.items():
    old_codes = list(info['old_codes'].keys())
    
    print(f"   {' + '.join([f'{c} {n}' for c, n in info['old_codes'].items()])}")
    print(f"   → {new_code} {info['new_name']} (merged {info['merge_year']})")
    
    # For each Year/Cat combination, sum old codes into new code
    for year in padron_hist['Year'].unique():
        for cat in padron_hist['Cat'].unique():
            # Get population from old codes
            old_pop = padron_hist[
                (padron_hist['Mun_Code'].isin(old_codes)) &
                (padron_hist['Year'] == year) &
                (padron_hist['Cat'] == cat)
            ]['Pop'].sum()
            
            # Add to new code (if old_pop > 0 or has data)
            if pd.notna(old_pop) and old_pop > 0:
                mask = (
                    (padron_hist['Mun_Code'] == new_code) &
                    (padron_hist['Year'] == year) &
                    (padron_hist['Cat'] == cat)
                )
                
                if mask.any():
                    # Add to existing value
                    current_pop = padron_hist.loc[mask, 'Pop'].iloc[0]
                    if pd.isna(current_pop):
                        padron_hist.loc[mask, 'Pop'] = old_pop
                    else:
                        padron_hist.loc[mask, 'Pop'] = current_pop + old_pop
                else:
                    # Create new row
                    new_row = pd.DataFrame({
                        'Mun_Code': [new_code],
                        'Mun': [info['new_name']],
                        'Cat': [cat],
                        'Year': [year],
                        'Pop': [old_pop]
                    })
                    padron_hist = pd.concat([padron_hist, new_row], ignore_index=True)
    
    # Remove old codes
    padron_hist = padron_hist[~padron_hist['Mun_Code'].isin(old_codes)].copy()
    
    # Show sample
    sample = padron_hist[
        (padron_hist['Mun_Code'] == new_code) & 
        (padron_hist['Cat'] == 'Total') & 
        (padron_hist['Year'].isin([1996, 2010, 2024]))
    ]
    print(f"   Sample (Total) after aggregation:")
    for _, row in sample.sort_values('Year').iterrows():
        pop_str = f"{row['Pop']:.0f}" if pd.notna(row['Pop']) else "NaN"
        print(f"      {int(row['Year'])}: {pop_str}")
    print()

# --- STEP 3: Validate ---
print("-"*70)
print("STEP 3: VALIDATION")
print("-"*70)

count_final = padron_hist['Mun_Code'].nunique()
INE_OFFICIAL_COUNT = 8132

print(f"\n   Before: {count_before}")
print(f"   Ghost codes removed: -2")
print(f"   Old merger codes removed: -4")
print(f"   New merger codes (already existed): +0")
print(f"   Final: {count_final}")

assert count_final == INE_OFFICIAL_COUNT, \
    f"Expected {INE_OFFICIAL_COUNT}, got {count_final}"
print(f"\n✓ Final count matches INE official figure ({INE_OFFICIAL_COUNT})")

# Sort and reset
padron_hist = padron_hist.sort_values(['Mun_Code', 'Year', 'Cat']).reset_index(drop=True)


CLEANING PADRON: GHOST CODES AND HISTORICAL MERGERS



Municipalities before cleaning: 8138

----------------------------------------------------------------------
STEP 1: REMOVING GHOST CODES
----------------------------------------------------------------------
   ✗ 12066 - Gatova: 84 records removed
   ✗ 17122 - Palmerola: 84 records removed

----------------------------------------------------------------------
STEP 2: AGGREGATING HISTORICAL MERGERS
----------------------------------------------------------------------
Old municipality data is added to existing new municipality codes:

   15026 Cesuras + 15063 Oza dos Ríos
   → 15902 Oza-Cesuras (merged 2013)
   Sample (Total) after aggregation:
      1996: 6057
      2010: 5407
      2024: 5151

   36011 Cerdedo + 36012 Cotobade
   → 36902 Cerdedo-Cotobade (merged 2016)
   Sample (Total) after aggregation:
      1996: 7894
      2010: 6726
      2024: 5705

----------------------------------------------------------------------
STEP 3: VALIDATION
--------------------------------------

### 🔍Missing Population Values: Diagnostic Summary
A number of rows in the dataset contain missing population values (NaN). These gaps are not limited to a single year and appear across multiple municipalities and categories, likely reflecting inconsistencies or omissions in the original INE records rather than errors introduced during processing. Since these missing values do not prevent the computation of population change rates—as long as each specific interval has valid data— they can be retained for now without compromising the subsequent analytical workflow. In later stages, these NaN entries can be explicitly handled, ignored, or imputed depending on the requirements of each analysis. For the moment, the dataset remains sufficiently robust to proceed with the population variation calculations.

### 🗂️Export rearranged clean file

In [20]:
PROCESSED_DIR = Path(
    r"/workspaces/rural-migration-land-use-spain/data/demography/processed"
)

output_padron_hist_clean = PROCESSED_DIR / "01_padron_clean_1996_2024.csv"

padron_hist.to_csv(
    output_padron_hist_clean,
    sep = ",",
    index = False,   # do not include the index column
    encoding = "latin1"   # recommended if I plan using the file in QGIS
)

## 📝Conclusion
The historical municipal population dataset from the INE has been successfully cleaned, standardized, and exported into a structured format suitable for analytical workflows. The preprocessing steps ensured consistent encoding, reliable numeric conversion, and the extraction of key geographic identifiers (CP and municipality names). As a result, the dataset is now ready for systematic analysis without the formatting issues typically present in raw administrative files.

#### ➡️Next Steps
The next notebook will focus on the analytical stage: computing population change across multiple temporal intervals, exploring annual and multi-year trends, and preparing the resulting indicators for potential spatial visualization in GIS environments. This will establish the basis for a reproducible pipeline linking raw statistical data with territorial analysis tools.